In [ ]:
import boto3

ec2 = boto3.client('ec2')
iam = boto3.client('iam')

In [ ]:
assume_role_policy_doc = """{
    "Version": "2012-10-17", 
    "Statement": [
        {
            "Action": "sts:AssumeRole", 
            "Effect": "Allow", 
            "Principal": {
                "Service": "ec2.amazonaws.com"
            }
        }
    ]
}"""

In [ ]:
response = iam.create_role(
    RoleName='S3-Super-User',
    AssumeRolePolicyDocument=assume_role_policy_doc
)

print response

In [ ]:
attach_policy_response = iam.attach_role_policy(
    RoleName='S3-Super-User',
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

print attach_policy_response

In [ ]:
detach_response = iam.detach_role_policy(
    RoleName='S3-Super-User',
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

print detach_response

In [ ]:
del_response = iam.delete_role(
    RoleName='S3-Super-User'
)
print del_response